In [149]:
import numpy as np
from smplx import SMPLH
import torch

data_path = "/home/meribejayson/Desktop/Projects/realistic-imu/data/TotalCapture_smplh/s5"
model_path = "/home/meribejayson/Desktop/Projects/realistic-imu/data/merged"

In [150]:
data = np.load(f"{data_path}/freestyle3_poses.npz")

for key in data.keys():
    print(key)

data["dmpls"].shape[1]

trans
gender
mocap_framerate
betas
dmpls
poses


8

In [151]:
smplh = smplx.create(model_path=model_path, gender='male', num_dmpls=8, num_betas=16)

In [152]:
# Load SMPL-H data from file
poses = torch.tensor(data['poses'], dtype=torch.float32)  # Shape: (n_frames, 156)
betas = torch.tensor(data['betas'], dtype=torch.float32).unsqueeze(0)  # Shape: (1, 10)
trans = torch.tensor(data['trans'], dtype=torch.float32)  # Shape: (n_frames, 3)
dmpls = torch.tensor(data['dmpls'], dtype=torch.float32)  # Shape: (n_frames, 10)

# Split poses into body and hand poses
body_pose = poses[:, 3:66]  # Body pose parameters (69 values)
global_orient = poses[:, :3]  # Global orientation (3 values)
left_hand_pose = poses[:, 66:111]  # Left hand pose (45 values)
right_hand_pose = poses[:, 111:]  # Right hand pose (45 values)

In [153]:
betas.shape

torch.Size([1, 16])

In [154]:

# Generate vertices for all frames
vertices = []
for i in range(poses.shape[0]):
    frame_verts = smplh(
        global_orient=global_orient[i].unsqueeze(0),  # Root orientation
        body_pose=body_pose[i].unsqueeze(0),  # Body joint rotations
        left_hand_pose=left_hand_pose[i].unsqueeze(0),  # Left hand articulation
        right_hand_pose=right_hand_pose[i].unsqueeze(0),  # Right hand articulation
        betas=betas,  # Shape coefficients
        dmpls=dmpls[i].unsqueeze(0),  # Dynamic deformations
        transl=trans[i].unsqueeze(0)  # Global translation
    ).vertices
    vertices.append(frame_verts)

# Convert list of vertices into NumPy array
vertices = torch.stack(vertices).detach().cpu().numpy()  # Shape: (n_frames, 10475, 3)
print("Vertices shape:", vertices.shape)

RuntimeError: size of dimension does not match previous size, operand 1, dim 0